# 목표와 설계 요약
      •	입력(고객): 맛 점수 4종(0–5)  body, richness, smoke, sweetness
      •	피처(모델): 오직 taste 4축으로 표준화 후 KNN(유클리드)
            :	랭킹은 맛 유사도만 반영 (가격/ABV 등은 표시/분류용으로만 사용)
      •	출력
        1.	기본 추천 리스트(거리 오름차순, Top-K)
        2.	Facet 뷰:
              •	price: 사분위 기반 버킷(price_bucket)
              •	cask group: casktype 등
              •	whisky_type
              •	bottling_year
              •	vintage 여부: vintage_clean이 N/A류가 아니면 Vintage
              •	rarity ≥ threshold: rarity_score가 임계치 이상만 별도 표시
      •	안정성: 컬럼 자동 탐지, 결측 중앙값 대체, 표준화 후 NaN/inf 제거, 그룹별 상위 N.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import gc

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
%matplotlib inline

!pip install koreanize-matplotlib
import koreanize_matplotlib

In [9]:
df = pd.read_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/whisky_clustered.csv')
df.columns = df.columns.str.lower()
print(df.shape)
df.head()

(1100, 58)


,name,main_info,whisky_type,volume(cl),alcohol(%),price(£),style_body_last,style_richness_last,style_smoke_last,style_sweetness_last,characters,food_paring,bottler,bottling_decade_clean,bottler_canon,is_independent,bottling_year,age,age_,age_int,is_nas,country,region,casktype,cask_group,colouring,vintage,vintage_clean,nose,palate,finish,reviews,_text_all,price_per_cl,alcohol_density,smoky_family,fruity_family,floral_family,herbal_family,sweet_family,nutty_family,spicy_family,woody_family,earthy_family,malty_family,creamy_family,wine_family,marine_family,other_family,rarity_scarcity,rarity_price_premium,rarity_distillery,rarity_vintage,rarity_score,cluster_taste,cluster_taste_name,pc1,pc2
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,blended scotch whisky,blended,70.0,46.0,34.25,3.0,3.0,0.0,3.0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee",NaN,NaN,NaN,thompson bros,1,NaN,NaN,NaN,17.584416,1,scotland,unknown_scotland,not specified,unknown,False,NaN,NaN,"Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...","Stunning stuff for the money, might well be my...",figs sultana cinnamon nutmeg toffee coffee plu...,0.489286,3220.0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0.399672,0.000750,0.0,0.000000,0.237533,1,Balanced,-0.019137,-0.615724
1,Johnnie Walker Blue Label,blended scotch whisky,blended,70.0,40.0,158.00,3.0,3.0,1.0,2.0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Octopus, Monkfish, Fondue",NaN,NaN,distillery bottling,0,NaN,NaN,NaN,17.584416,1,scotland,unknown_scotland,not specified,unknown,True,NaN,NaN,"Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,Un elixir. || Excellent Whisky. Smooth on the ...,rose hazelnut honey smoke dark chocolate tobac...,2.257143,2800.0,0,0,1,0,1,1,1,1,1,1,0,0,0,1,0.000000,0.009766,0.0,0.000000,0.005793,1,Balanced,-0.778212,1.594287
2,The Big Ben,blended scotch whisky,blended,50.0,40.0,23.50,3.1,3.0,0.8,3.0,no description,NaN,NaN,NaN,distillery bottling,0,NaN,NaN,NaN,17.584416,1,scotland,unknown_scotland,not specified,unknown,True,NaN,NaN,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,Very pleased with the whiskey. Highly recommend,no description medium weight malty nose with h...,0.470000,2000.0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0.000000,0.273168,0.0,0.000000,0.162045,1,Balanced,0.081790,0.161437
3,J&B Rare,blended scotch whisky,blended,70.0,40.0,25.25,2.8,2.7,0.2,2.8,no description,NaN,NaN,NaN,distillery bottling,0,NaN,NaN,NaN,17.584416,1,scotland,unknown_scotland,not specified,unknown,True,NaN,NaN,no description,no description,no description,Very nice || Was delighted to find this whisky...,no description no description no description n...,0.360714,2800.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000095,0.0,0.000000,0.000056,1,Balanced,-0.689116,-0.306851
4,Dimple 15 Year OldLitre,blended scotch whisky,blended,100.0,43.0,69.95,3.0,3.0,0.0,3.0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",NaN,NaN,NaN,distillery bottling,0,NaN,15 year old,15.0,15.000000,0,scotland,unknown_scotland,not specified,unknown,Unknown,NaN,NaN,no description,no description,no description,NaN,caramel cinnamon vanilla dark chocolate no des...,0.699500,4300.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0.333333,0.003351,0.0,0.312207,0.323193,1,Balanced,-0.019137,-0.615724


In [6]:
# =========================================================
# 06_recommendation_system.ipynb (taste-first + facet views)
# - 고객 입력: body, richness, smoke, sweetness (0~5)
# - KNN 유사도 추천
# - 결과를 price/cask/whisky_type/bottling_year/vintage/rarity로 따로 보여줌
# =========================================================

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# -----------------------------
# 0) Column Resolver (유연 매핑)
# -----------------------------
def _resolve(df, patterns, required=False):
    cols_map = {c.lower().strip(): c for c in df.columns}
    for pat in patterns:
        # exact
        low = pat.lower()
        if low in cols_map:
            return cols_map[low]
        # regex
        rx = re.compile(pat, re.I)
        for k, orig in cols_map.items():
            if rx.search(k):
                return orig
    if required:
        raise KeyError(f"Column not found for patterns: {patterns}")
    return None

def _resolve_taste_cols(df):
    return dict(
        body=_resolve(df, [r"^style_body_last$", r"^style_body$", r"\bbody\b"], required=True),
        richness=_resolve(df, [r"^style_richness_last$", r"^style_richness$", r"\brichness\b"], required=True),
        smoke=_resolve(df, [r"^style_smoke_last$", r"^style_smoke$", r"\bsmoke|peat"], required=True),
        sweetness=_resolve(df, [r"^style_sweetness_last$", r"^style_sweetness$", r"\bsweet(ness)?\b"], required=True),
    )

def _vintage_flag_series(df, vintage_col):
    if vintage_col is None or vintage_col not in df.columns:
        return pd.Series(False, index=df.index, name="is_vintage")
    s = df[vintage_col].astype(str).str.strip().str.lower()
    na_like = {"", "na", "n/a", "none", "nan", "null"}
    return (~s.isin(na_like)) & (s != "0")

# ---------------------------------------------
# 1) 준비: 컬럼 자동 인식 + 피쳐 행렬 생성
# ---------------------------------------------
# 필수 강력추천 후보 컬럼들
taste_map = _resolve_taste_cols(df)
col_price   = _resolve(df, [r"^price", r"price.*£", r"price_?usd", r"price_?krw"])
col_abv     = _resolve(df, [r"^alcohol\(%\)$", r"\babv\b", r"alcohol", r"alc[%]?" ])
col_cask    = _resolve(df, [r"^cask[_ ]?type$", r"\bcask(type)?\b", r"cask.*group"])
col_type    = _resolve(df, [r"^whisky[_ ]?type$", r"\b(type|category|style)$", r"whiskey[_ ]?type"])
col_year    = _resolve(df, [r"\bbottling[_ ]?year\b", r"\byear\b", r"vintage_year"])
col_vintage = _resolve(df, [r"^vintage_clean$", r"\bvintage\b"])
col_rarity  = _resolve(df, [r"^rarity_score$", r"^rare_score$", r"\brarity\b", r"rare_index"])

# 사용 피쳐(기본: 풍미 4종만) — *가격/ABV는 유사도 계산에는 기본 미포함*
taste_cols = [taste_map["body"], taste_map["richness"], taste_map["smoke"], taste_map["sweetness"]]

df_feat = df.copy()

# 숫자화 + 결측치 중앙값 대체
for c in taste_cols:
    df_feat[c] = pd.to_numeric(df_feat[c], errors="coerce")
med = df_feat[taste_cols].median(numeric_only=True)
df_feat[taste_cols] = df_feat[taste_cols].fillna(med).clip(lower=0, upper=5)

# 스케일링 + KNN (taste 전용)
scaler = StandardScaler()
X = scaler.fit_transform(df_feat[taste_cols])
knn  = NearestNeighbors(metric="euclidean", n_neighbors=100)  # 넉넉히(후속 필터 대비)
knn.fit(X)

# vintage 플래그 칼럼 생성
df_feat["is_vintage"] = _vintage_flag_series(df_feat, col_vintage)

# bottling_year 정수화
if col_year:
    df_feat[col_year] = pd.to_numeric(df_feat[col_year], errors="coerce").astype("Int64")

# 가격 버킷 유틸
def _price_bucket_series(s: pd.Series, mode="quantile"):
    s = pd.to_numeric(s, errors="coerce")
    if s.notna().sum() == 0:
        return pd.Series("Unknown", index=s.index, name="price_bucket")
    if mode == "quantile":
        # 4분위 버킷
        try:
            q = pd.qcut(s, 4, labels=["Q1-Low","Q2","Q3","Q4-High"])
        except ValueError:
            # 값이 적어 qcut 실패 시 균등 bin
            q = pd.cut(s, bins=4, labels=["Q1-Low","Q2","Q3","Q4-High"])
        return q.astype(str).rename("price_bucket")
    # 명시 bins 예: bins=[0,40,80,150,1e9]
    return s

if col_price:
    df_feat["price_bucket"] = _price_bucket_series(df_feat[col_price], mode="quantile")
else:
    df_feat["price_bucket"] = "Unknown"

# ---------------------------------------------
# 2) 고객 입력 기반 추천 함수 (이름 불필요)
# ---------------------------------------------
def recommend_from_profile(
    *,
    body: float, richness: float, smoke: float, sweetness: float,
    top_k: int = 60,
    weights: dict | None = None,
    extra_filters: dict | None = None,  # 사전 필터(선택)
):
    """
    - 입력: 맛 점수(0~5). (자동 클리핑)
    - 출력: 유사도 근접 순 정렬 후보 top_k (distance 포함)
    - weights: {"body":1.0,"richness":1.0,"smoke":1.0,"sweetness":1.0}
               taste 축 가중치 (선택)
    - extra_filters 예시:
      {"country": ["Scotland","Japan"], "region": ["Islay"], "cluster_taste": [2,5]}
    """
    # 1) 사전 필터 적용(있는 경우)
    base = df_feat
    if extra_filters:
        for k, allow in extra_filters.items():
            if k in base.columns:
                base = base[base[k].isin(allow)]
    if len(base) == 0:
        raise ValueError("사전 필터 결과가 비어있습니다. 조건을 완화하세요.")

    # 2) 쿼리 벡터 구성 (taste 4축)
    q = np.array([
        np.clip(float(body), 0, 5),
        np.clip(float(richness), 0, 5),
        np.clip(float(smoke), 0, 5),
        np.clip(float(sweetness), 0, 5),
    ]).reshape(1, -1)

    # 3) 가중치 적용 (선택)
    if weights is not None:
        w = np.array([
            float(weights.get("body",1.0)),
            float(weights.get("richness",1.0)),
            float(weights.get("smoke",1.0)),
            float(weights.get("sweetness",1.0)),
        ]).reshape(1, -1)
        q = q * w

        # 데이터 쪽도 동일 가중 적용
        X_used = X * w
    else:
        X_used = X

    # 4) 후보 인덱스 맵핑(사전 필터가 있었다면 해당 subset의 인덱스만)
    idx_map = base.index.to_numpy()
    X_sub = X_used[idx_map]

    # 5) 스케일에 맞춘 쿼리 변환
    #   - 주의: scaler는 taste_cols 기준이므로 q를 데이터 스케일로 변환해야 한다.
    #   - trick: 원래 데이터 중앙값 기반으로 inverse_transform 할 필요 없이,
    #            scaler.mean_ / scale_를 사용해 표준화 동일 적용
    q_std = (q - scaler.mean_) / scaler.scale_

    # 6) KNN 탐색
    nns = min(top_k, len(X_sub))
    dist, nn_idx = NearestNeighbors(metric="euclidean", n_neighbors=nns).fit(X_sub).kneighbors(q_std, n_neighbors=nns)

    res = df_feat.loc[idx_map[nn_idx[0]]].copy()
    res["distance"] = dist[0]
    # 표시 컬럼 정리
    show_cols = ["name","country","region","cluster_taste"] if "cluster_taste" in res.columns else ["name","country","region"]
    if col_price: show_cols.append(col_price)
    if col_abv:   show_cols.append(col_abv)
    if col_cask:  show_cols.append(col_cask)
    if col_type:  show_cols.append(col_type)
    if col_year:  show_cols.append(col_year)
    show_cols += ["is_vintage", "distance"]
    show_cols = [c for c in show_cols if c in res.columns]
    return res[show_cols].reset_index(drop=True)

# ---------------------------------------------
# 3) Facet Views: 결과를 '따로' 보여주기
# ---------------------------------------------
def _topn_by_group(df_in, group_col, topn=5, max_groups=8, sort_col="distance"):
    if group_col not in df_in.columns:
        return pd.DataFrame(columns=[group_col] + list(df_in.columns))
    tmp = df_in.copy()
    tmp = tmp.sort_values(sort_col, ascending=True)
    # 그룹 내 상위 N
    top = tmp.groupby(group_col, group_keys=True).head(topn)
    # 그룹 수 제한 (distance 평균 기준 상위 그룹)
    if max_groups is not None:
        keep_groups = (
            top.groupby(group_col)["distance"]
            .mean()
            .sort_values()
            .head(max_groups)
            .index
        )
        top = top[top[group_col].isin(keep_groups)]
    return top

def facet_views(
    base_reco: pd.DataFrame,
    *,
    price_mode="quantile",  # 또는 bins=[0,40,80,150,9e9]
    price_topn=5, cask_topn=5, type_topn=5, year_topn=5,
    rare_threshold: float | None = None,
    max_groups=8
):
    out = {}

    # 3-1) price별
    if col_price:
        if "price_bucket" not in base_reco.columns and "price_bucket" in df_feat.columns:
            # 원 데이터의 버킷을 붙임
            base_reco = base_reco.merge(df_feat[["name","price_bucket"]], on="name", how="left")
        out["by_price_bucket"] = _topn_by_group(base_reco, "price_bucket", topn=price_topn, max_groups=max_groups)
    else:
        out["by_price_bucket"] = pd.DataFrame(columns=base_reco.columns)

    # 3-2) cask group 별
    out["by_cask_group"] = _topn_by_group(base_reco, col_cask if col_cask else "_NO_CASK_", topn=cask_topn, max_groups=max_groups)

    # 3-3) whisky_type 별
    out["by_whisky_type"] = _topn_by_group(base_reco, col_type if col_type else "_NO_TYPE_", topn=type_topn, max_groups=max_groups)

    # 3-4) bottling_year 별 (최대 그룹 제한)
    if col_year:
        out["by_bottling_year"] = _topn_by_group(base_reco, col_year, topn=year_topn, max_groups=max_groups)
    else:
        out["by_bottling_year"] = pd.DataFrame(columns=base_reco.columns)

    # 3-5) vintage 여부 별
    out["vintage_yes"] = base_reco[base_reco["is_vintage"] == True].sort_values("distance").head(10)
    out["vintage_no"]  = base_reco[base_reco["is_vintage"] == False].sort_values("distance").head(10)

    # 3-6) rarity threshold 이상
    if (rare_threshold is not None) and (col_rarity in base_reco.columns or col_rarity in df_feat.columns):
        if col_rarity not in base_reco.columns and col_rarity in df_feat.columns:
            base_reco = base_reco.merge(df_feat[["name", col_rarity]], on="name", how="left")
        out["rarity_over_threshold"] = (
            base_reco[pd.to_numeric(base_reco[col_rarity], errors="coerce") >= float(rare_threshold)]
            .sort_values("distance")
            .head(15)
        )
    else:
        out["rarity_over_threshold"] = pd.DataFrame(columns=base_reco.columns)

    return out

# ---------------------------------------------
# 4) 사용 예시 (고객 입력 → 추천 → facet별로 따로 보여주기)
# ---------------------------------------------
# 예)  smoke↑ body↑ richness↑ sweetness↓
reco = recommend_from_profile(body=4.3, richness=4.0, smoke=5.0, sweetness=1.8, top_k=80)

# 기본 상위 20
display(reco.head(20))

# Facet별 분리 (희귀점수 0.75 이상만 따로)
views = facet_views(
    reco,
    price_mode="quantile",
    price_topn=5, cask_topn=5, type_topn=5, year_topn=5,
    rare_threshold=0.75,   # rarity_score 없으면 빈 DF로 반환
    max_groups=8
)

print("\n=== Price Bucket ===")
display(views["by_price_bucket"])
print("\n=== By Cask Group ===")
display(views["by_cask_group"])
print("\n=== By Whisky Type ===")
display(views["by_whisky_type"])
print("\n=== By Bottling Year ===")
display(views["by_bottling_year"])
print("\n=== Vintage: YES ===")
display(views["vintage_yes"])
print("\n=== Vintage: NO ===")
display(views["vintage_no"])
print("\n=== Rarity ≥ Threshold ===")
display(views["rarity_over_threshold"])

,name,country,region,cluster_taste,price(£),alcohol(%),casktype,whisky_type,bottling_year,is_vintage,distance
0,Big Peat Feis Ile 2025,scotland,unknown_scotland,3,63.75,50.0,not specified,blended malt,<NA>,False,1.407474
1,Big Peat Blended Malt Christmas Edition,scotland,islay,3,51.75,52.8,not specified,blended malt,<NA>,False,1.994940
2,Blended Islay 20109 Year Old Whisky Trail Retr...,scotland,islay,3,77.50,59.8,not specified,blended,<NA>,False,2.576344
3,Tomintoul Peated,scotland,speyside,3,37.25,40.0,not specified,single malt,<NA>,False,2.860494
4,Compass Box Flaming Heart 25th Anniversary Edi...,scotland,unknown_scotland,3,130.00,48.9,not specified,blended malt,<NA>,False,2.860494
5,Compass Box Duality,scotland,unknown_scotland,3,176.00,52.0,not specified,blended malt,<NA>,False,2.860494
6,Elements of Islay Fireside,scotland,islay,3,82.50,54.5,sherry hogshead,blended malt,<NA>,False,2.860494
7,Hakushu 18 Year Old PeatedTsukuriwake 2024,japan,unknown_japan,3,1250.00,48.0,not specified,single malt,<NA>,False,2.919358
8,Big Peat,scotland,islay,3,38.75,46.0,not specified,other,<NA>,False,3.021031
9,Big Peat 12 Year Old,scotland,unknown_scotland,3,53.25,46.0,not specified,blended malt,<NA>,False,3.554428



=== Price Bucket ===


,name,country,region,cluster_taste,price(£),alcohol(%),casktype,whisky_type,bottling_year,is_vintage,distance,price_bucket
0,Big Peat Feis Ile 2025,scotland,unknown_scotland,3,63.75,50.0,not specified,blended malt,<NA>,False,1.407474,Q2
1,Big Peat Blended Malt Christmas Edition,scotland,islay,3,51.75,52.8,not specified,blended malt,<NA>,False,1.994940,Q2
2,Blended Islay 20109 Year Old Whisky Trail Retr...,scotland,islay,3,77.50,59.8,not specified,blended,<NA>,False,2.576344,Q2
3,Tomintoul Peated,scotland,speyside,3,37.25,40.0,not specified,single malt,<NA>,False,2.860494,Q1-Low
4,Compass Box Flaming Heart 25th Anniversary Edi...,scotland,unknown_scotland,3,130.00,48.9,not specified,blended malt,<NA>,False,2.860494,Q3
5,Compass Box Duality,scotland,unknown_scotland,3,176.00,52.0,not specified,blended malt,<NA>,False,2.860494,Q3
6,Elements of Islay Fireside,scotland,islay,3,82.50,54.5,sherry hogshead,blended malt,<NA>,False,2.860494,Q2
7,Hakushu 18 Year Old PeatedTsukuriwake 2024,japan,unknown_japan,3,1250.00,48.0,not specified,single malt,<NA>,False,2.919358,Q4-High
8,Big Peat,scotland,islay,3,38.75,46.0,not specified,other,<NA>,False,3.021031,Q1-Low
9,Big Peat 12 Year Old,scotland,unknown_scotland,3,53.25,46.0,not specified,blended malt,<NA>,False,3.554428,Q2



=== By Cask Group ===


,name,country,region,cluster_taste,price(£),alcohol(%),casktype,whisky_type,bottling_year,is_vintage,distance,price_bucket
0,Big Peat Feis Ile 2025,scotland,unknown_scotland,3,63.75,50.0,not specified,blended malt,<NA>,False,1.407474,Q2
1,Big Peat Blended Malt Christmas Edition,scotland,islay,3,51.75,52.8,not specified,blended malt,<NA>,False,1.994940,Q2
2,Blended Islay 20109 Year Old Whisky Trail Retr...,scotland,islay,3,77.50,59.8,not specified,blended,<NA>,False,2.576344,Q2
3,Tomintoul Peated,scotland,speyside,3,37.25,40.0,not specified,single malt,<NA>,False,2.860494,Q1-Low
4,Compass Box Flaming Heart 25th Anniversary Edi...,scotland,unknown_scotland,3,130.00,48.9,not specified,blended malt,<NA>,False,2.860494,Q3
6,Elements of Islay Fireside,scotland,islay,3,82.50,54.5,sherry hogshead,blended malt,<NA>,False,2.860494,Q2
10,Benriach 199427 Year Old Oloroso Cask #2056 UK...,scotland,speyside,3,373.00,52.8,oloroso sherry puncheon,single malt,<NA>,False,3.567253,Q4-High
11,Karuizawa 1999Artist #13 Spirits Shop Selectio...,japan,unknown_japan,3,4000.00,60.0,sherry butt,other,<NA>,False,3.567253,Q4-High
12,Inchgower 199826 Year Old Cask #11274 Connoiss...,scotland,speyside,3,259.00,54.8,refill sherry butt,single malt,<NA>,False,3.567253,Q4-High
14,Nagahama 20183 Year Old Bourbon Heavily Peated...,japan,unknown_japan,3,210.00,61.2,bourbon barrel,single malt,<NA>,False,3.650927,Q3


...

,name,country,region,cluster_taste,price(£),alcohol(%),casktype,whisky_type,bottling_year,is_vintage,distance,price_bucket



=== Vintage: NO ===


,name,country,region,cluster_taste,price(£),alcohol(%),casktype,whisky_type,bottling_year,is_vintage,distance,price_bucket
0,Big Peat Feis Ile 2025,scotland,unknown_scotland,3,63.75,50.0,not specified,blended malt,<NA>,False,1.407474,Q2
1,Big Peat Blended Malt Christmas Edition,scotland,islay,3,51.75,52.8,not specified,blended malt,<NA>,False,1.994940,Q2
2,Blended Islay 20109 Year Old Whisky Trail Retr...,scotland,islay,3,77.50,59.8,not specified,blended,<NA>,False,2.576344,Q2
3,Tomintoul Peated,scotland,speyside,3,37.25,40.0,not specified,single malt,<NA>,False,2.860494,Q1-Low
4,Compass Box Flaming Heart 25th Anniversary Edi...,scotland,unknown_scotland,3,130.00,48.9,not specified,blended malt,<NA>,False,2.860494,Q3
5,Compass Box Duality,scotland,unknown_scotland,3,176.00,52.0,not specified,blended malt,<NA>,False,2.860494,Q3
6,Elements of Islay Fireside,scotland,islay,3,82.50,54.5,sherry hogshead,blended malt,<NA>,False,2.860494,Q2
7,Hakushu 18 Year Old PeatedTsukuriwake 2024,japan,unknown_japan,3,1250.00,48.0,not specified,single malt,<NA>,False,2.919358,Q4-High
8,Big Peat,scotland,islay,3,38.75,46.0,not specified,other,<NA>,False,3.021031,Q1-Low
9,Big Peat 12 Year Old,scotland,unknown_scotland,3,53.25,46.0,not specified,blended malt,<NA>,False,3.554428,Q2



=== Rarity ≥ Threshold ===


,name,country,region,cluster_taste,price(£),alcohol(%),casktype,whisky_type,bottling_year,is_vintage,distance,price_bucket,rarity_score
50,SMWS 116.10 (Yoichi)1987 18 Year Old,japan,unknown_japan,3,5000.0,52.2,not specified,single malt,<NA>,False,4.457683,Q4-High,0.810388
80,Hakushu 1981Vintage Malt Sherry Cask Bot.2004,japan,unknown_japan,4,10000.0,56.0,not specified,single malt,2004,False,4.505190,Q4-High,0.772424
98,Yamazaki 1986Vintage Malt Bot.2005,japan,unknown_japan,3,15000.0,56.0,not specified,single malt,2005,False,5.044163,Q4-High,0.763783
100,Chichibu 20159 Year Old Cask #4415 Berry Bros ...,japan,unknown_japan,5,550.0,65.2,not specified,single malt,<NA>,False,5.166634,Q4-High,0.752687
